In [1]:
#imports

import os
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
from numpy import reshape
import re
%matplotlib inline

In [3]:
os.chdir("/Users/andrewcaffey/Documents/Projects/Data/PCPP/parts/memory/parts/")
part_list = []
comments = []
for i in os.listdir(os.getcwd()):
    a = open(i, 'r')
    #print a.read()
    b = BeautifulSoup(a)

    #basic info
    if b.find('h4', attrs={'class':'kind'}) != None:
        kind = b.find('h4', attrs={'class':'kind'}).text
        part_name = b.find('h1', attrs={'class':'name'}).text
        link = b.find('input', attrs={'name':'url'})['value']
        info_dict = {'Kind':kind, 'Name':part_name, 'Link': link}

        #prices
        if b.find_all('td', attrs={'class':'base'}) != None:
            price_list = b.find_all('td', attrs={'class':'base'})
            price_list = [float(x.text.strip('$')) for x in price_list]
            #average_price = sum(price_list)/len(price_list)
            price_dict = {'Prices':price_list,}

        #specs
        spec_labels = b.find('div', attrs={'class':'specs block'}).find_all('h4')
        spec_labels = [x.contents[0] for x in spec_labels]
        spec_values = str(b.find('div', attrs={'class':'specs block'}))

        vals = [x.strip().split('</h4>')[1].strip('\n').strip() for x in spec_values.split("<h4>")[1:]]
        vals[-1] = vals[-1].split('\n')[0]
        spec_values = vals

        spec_values = spec_values[0:len(spec_labels)+1]
        spec_dict = {spec_label:spec_value for spec_label, spec_value in zip(spec_labels,spec_values)}

        part_dict = dict(spec_dict.items() + info_dict.items() + price_dict.items())
        part_list.append(part_dict)

        #reviews
    reviews = b.find('div', attrs={'class':'part-reviews'})
    if reviews != None:
        reviews = reviews.find_all('div',attrs={'class':'part-review-block'})
        star_list = [len(reviews[x].find('ul',attrs={'class':'stars'}).find_all('li',attrs={'class':'full-star'})) for x in range(len(reviews))]

        comment_text_list = b.find_all('div', attrs={'class':'comment-message markdown'})
        comment_text_list = [comment_text_list[x].find_all('p') for x in range(len(comment_text_list))]

        comment_text_list_clean = []
        for i, x in enumerate(comment_text_list):
            comment = ""
            for y in x:
                try:
                    comment += y.contents[0] + " "
                except:
                    pass
            comment_text_list_clean.append(comment)

        

        review = zip(star_list, comment_text_list_clean)
        
        comments.append(review)

    a.close()
    b.decompose()

In [4]:
df = pd.DataFrame(part_list)

In [6]:
#standard

#average price
df['avg'] = [sum(x)/len(x) if len(x)>0 else 0 for x in df.Prices]

#short_link
df['short_link'] = [x.split('/')[-1] for x in df.Link]

In [7]:
#CAS
df["CAS"] = [int(x) for x in df['CAS Latency']]

In [8]:
#price per GB
df['ppgb'] = [float(x.strip('$')) if type(x)==str else 0 for x in df['Price/GB']]

In [9]:
#ddr3/4

df['is_ddr4'] = [True if 'DDR4' in x else False for x in df.Speed]
df['ddr3_speed'] = [int(x.split('-')[1]) if 'DDR3' in x else None for x in df.Speed]
df['ddr4_speed'] = [int(x.split('-')[1]) if 'DDR4' in x else None for x in df.Speed]
df['ddr_speed'] = [int(x.split('-')[1]) for x in df.Speed]

In [15]:
df['ddr_type'] = ['DDR4' if 'DDR4' in x else 'DDR3' if 'DDR3' in x  else 'DDR2' if 'DDR2' in x else 'other' for x in df['Speed']]

In [14]:
df.ddr_type.value_counts()

DDR3    1191
DDR4     461
DDR2      48
Name: ddr_type, dtype: int64

In [10]:
#dimm
df['dimm'] = [int(x.split('-')[0]) for x in df.Type]

In [11]:
#module size and count

df['size_gb'] = [float(x.split('(')[0].strip().strip('GB')) if type(x)==str else 0 for x in df.Size]
df['mod_count'] = [float(x.split('(')[1].split('x')[0].strip()) if type(x)==str else 0 for x in df.Size]
df['mod_size'] = [x/y for x,y in zip(df.size_gb,df.mod_count)]

In [12]:
#voltage
df['voltage'] = [float(x.strip('V')) for x in df.Voltage]

In [16]:
#save df to local csv file
df.to_csv('memory_csv_FINAL.csv', encoding='utf-8', index=False)

In [50]:
stars = [x[0][0] for x in comments[:]]
text = [x[0][1] for x in comments[:]]
memory_part_reviews = pd.DataFrame({'Stars':stars, 'Comments':text, 'Type':'memory'})
memory_part_reviews.to_csv('memory_part_reviews.csv', encoding='utf-8', index=False)

In [2]:
df = pd.read_csv('memory_csv.csv')